In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# load model
from tensorflow.keras.models import load_model
savedModel=load_model('/content/drive/MyDrive/UIUC_Propeller2.keras')
savedModel.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 128)               21120     
                                                                 
 dense_33 (Dense)            (None, 128)               16512     
                                                                 
 dense_34 (Dense)            (None, 64)                8256      
                                                                 
 dense_35 (Dense)            (None, 64)                4160      
                                                                 
 dense_36 (Dense)            (None, 32)                2080      
                                                                 
 dense_37 (Dense)            (None, 32)                1056      
                                                                 
 dense_38 (Dense)            (None, 1)                

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/volume1_exp.csv")
df = df.drop(["PropName","B"],axis=1)

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

ct = make_column_transformer(
    (MinMaxScaler(), df.columns[2:-1].tolist()),
    (OneHotEncoder(handle_unknown="ignore",sparse=False), df.columns[0:2].tolist())
)

X = df.drop("eta", axis=1)
y = df["eta"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


ct.fit(X_train)


X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
single = ct.transform(X_test)
test_data = {
    "y_test":y_test.values,
    "y_pred":np.round(savedModel.predict(single),3).flatten()
}
df2 = pd.DataFrame(test_data)
df2

103/103 [==============================] - 0s 3ms/step


,y_test,y_pred
0,0.000,-0.001
1,0.702,0.704
2,0.644,0.645
3,0.403,0.390
4,0.224,0.250
...,...,...
3286,0.673,0.676
3287,-0.359,-0.355
3288,0.505,0.507
3289,0.564,0.568


In [ ]:
model_loss, model_mae = savedModel.evaluate(X_test_normal, y_test)

103/103 [==============================] - 1s 3ms/step - loss: 0.0106 - mae: 0.0106
